## Creation and modifcation of Rasters  
To study the evolution of the forecast for the 18 of may of 2024, we use era 5 and ecmwf data to produce rasters, for a future visual analysis.  

In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import scipy 
import pandas as pd
import os
import matplotlib.dates as mdates
import geopandas as gpd
import rasterio
from rasterio.transform import from_origin
from geocube.api.core import make_geocube
from geocube.rasterize import rasterize_points_radial # Different choices here
from geocube.rasterize import rasterize_points_griddata # Different choices here
from functools import partial
from random import sample 

In [2]:
#change this address to your project location
path='g:\\Mi unidad\\Universitat\\[SoSe24] Fundamentals of Earth System data processing\\ESDP_project\\Weather_forecast_case_stuty'
os.chdir(path)

In [3]:

data_path = [ 'data\\perturbed\\'+s  for s in os.listdir('data\\perturbed')]
data_path = sorted(data_path, reverse=True)

In [4]:
#this code is for extract raster datafiles from the netcdf, of the mean forecast for the 18 of may, and save it in the raster file

for file in data_path[0:]:
    
    data = xr.open_dataset(file)
    data.close()

    #the index is to obtain all the values htat have the 18 of may
    Mays18_index = (data['time']==pd.to_datetime('2024-05-18T00:00:00.000000000'))

    data = data.sel(time= Mays18_index)
    data = data.mean(dim='number',skipna= True)

    #then we convert the netcdf file to data frame
    df = data.to_dataframe(dim_order=None)
    df = df.reset_index()
    df = df[['longitude','latitude',"tp"]]
    
    #then we convert the  data frame to geodataframe

    df['longitude'] = np.where(df['longitude'] > 180,df['longitude'] - 360,df['longitude'])
    gdf = gpd.GeoDataFrame(df, 
                        geometry=gpd.points_from_xy(df.longitude, df.latitude), 
                        crs=4326)
    
    #to finally convert it to raster

    geo_grid = make_geocube(
        vector_data=gdf,
        measurements=['tp'],
        resolution=0.1, # degrees
        rasterize_function=partial(rasterize_points_griddata, method="cubic", filter_nan=True),
        
    )

    #finally we save the raster

    #we select the name of the file
    input_string = file

    # Split by the last backslash
    filename_with_extension = input_string.split('\\')[-1]

    # Split by the first dot
    desired_part = filename_with_extension.split('.')[0]

    #now we save with the name extracted
    geo_grid.rio.to_raster(f'data/raster/{desired_part}.tif')

In [5]:
#We have to do the same to the real era 5 reanalisis data

data = xr.open_dataset('data\\era5\\era5_new_units.nc')

data.close()

#the index is to obtain all the values htat have the 18 of may
Mays18_index = (data['time']==pd.to_datetime('2024-05-18T00:00:00.000000000'))
data = data.sel(time= Mays18_index)

#then we convert the netcdf file to data frame
df = data.to_dataframe(dim_order=None)
df = df.reset_index()
df = df[['longitude','latitude',"tp"]]
    
#then we convert the  data frame to geodataframe

df['longitude'] = np.where(df['longitude'] > 180,df['longitude'] - 360,df['longitude'])
gdf = gpd.GeoDataFrame(df, 
                    geometry=gpd.points_from_xy(df.longitude, df.latitude), 
                    crs=4326)
    
#to finally convert it to raster

geo_grid = make_geocube(
    vector_data=gdf,
    measurements=['tp'],
    resolution=0.1, # degrees
    rasterize_function=partial(rasterize_points_griddata, method="cubic", filter_nan=True),
        
    )

#finally we save the raster
geo_grid.rio.to_raster(f'data/raster/era5_new_units.tif')